# 🧾 Parse the LLM Output String into a Python Dictionary

## 💡 Introduction
Often, the output from an **LLM (Large Language Model)** comes back as a **string**.  
However, to use it effectively in your application, you might need to **parse** that string into a structured format — like a **Python dictionary**.

---

## ⚙️ Why Parsing Is Important
Parsing the LLM output allows you to:
- Work with the data programmatically (e.g., access fields by keys).  
- Validate and manipulate the results easily.  
- Integrate LLM responses into larger applications and workflows.

---

# without parse

In [ ]:
!pip install openai python-dotenv


In [ ]:
from openai import OpenAI
import openai
import os
import os
from dotenv import load_dotenv
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

client = OpenAI(
    api_key=groq_api_key,
    base_url="https://api.groq.com/openai/v1"
)

In [ ]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template =ChatPromptTemplate.from_template(review_template)



In [ ]:
print (prompt_template)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'), additional_kwargs={})]


In [ ]:
from langchain_community.chat_models import ChatOpenAI

chat =ChatOpenAI(model=llm_model,temperature=0,api_key=groq_api_key,    base_url="https://api.groq.com/openai/v1")


In [ ]:
messages=prompt_template.format_messages(text=customer_review)

In [ ]:
response = chat.invoke(messages)
print(response.content)


```json
{
  "gift": true,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


In [ ]:
type(response.content)
# it is string

str

#  Parse the LLM Output String into a Python Dictionary


In [ ]:

from langchain.output_parsers import ResponseSchema


gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")


response_schemas = [gift_schema,
                    delivery_days_schema,
                    price_value_schema]

In [ ]:
from langchain.output_parsers import StructuredOutputParser

output_parser=StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions =output_parser.get_format_instructions()

In [ ]:
print (format_instructions)

T


In [ ]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review,
                                format_instructions=format_instructions)

In [ ]:
response =chat.invoke(messages)

In [ ]:
print (response.content)

```json
{
	"gift": "True",
	"delivery_days": "2",
	"price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


In [ ]:
output_dict = output_parser.parse(response.content)
output_dict

{'gift': 'True',
 'delivery_days': '2',
 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}